In [1]:
import json

import tifffile as tf
from cffi import FFI

In [2]:
file_path = './origin/2025_08_31_U2280_Electrodes_GePP_HR_3um_001.ssc'

with open(file_path, encoding='latin-1') as f:
    letto = f.read()
    print(letto)

[METADATA]
Acc. Voltage=0.0kV
Active Mode=10kv_HC_CO_Work2
Average Count=1
Average Type=Moving Frame Averaging
Date=2025/08/31
Detector=SE2
FoV=400.0µm
Focus/WD=7.098mm
Pixel Dwelltime=2µs
Point Average=1
Position U=1.0370mm
Position V=2.4965mm
Probe Current=20.5pA
Project=%UserRoot%Project\New.DSK
Stage R=0.0000mm
Stage T=0.0000mm
Stage X=18.0453mm
Stage Y=-39.3926mm
Stage Z=25.6202mm
Step Size=0.7812µm
Time=21:25:07
User=Giubertoni
Write field=200µm
[SLOWSCAN]
Bitmap=2025_08_31 U2280_Electrodes_GePP_HR_3um_001.bmp
DataZone=Damiano_SEM
LocalOriginUV=1.036989,2.496478
LowerLeftUV=0.836989,2.296478
RotationUV=0
ScanSignal=A
ScanSource=SEM
StageUV=1.036989,2.496478
UpperRightUV=1.236989,2.696478



In [3]:
from read_external_file import read_external_file

json_str = json.dumps(read_external_file(file_path))
print(json_str)

{"Acc. Voltage": 0.0, "Active Mode": "10kv_HC_CO_Work2", "Average Count": "1", "Average Type": "Moving Frame Averaging", "Date": "2025/08/31", "Detector": "SE2", "FoV": 0.0004, "Focus/WD": 0.007098, "Pixel Dwelltime": 2e-06, "Point Average": "1", "Position U": 0.001037, "Position V": 0.0024965, "Probe Current": 2.05e-11, "Project": "%UserRoot%Project\\New.DSK", "Stage R": 0.0, "Stage T": 0.0, "Stage X": 0.0180453, "Stage Y": -0.0393926, "Stage Z": 0.0256202, "Step Size": 7.812e-07, "Time": "21:25:07", "User": "Giubertoni", "Write field": 0.0002, "Bitmap": "2025_08_31 U2280_Electrodes_GePP_HR_3um_001.bmp", "DataZone": "Damiano_SEM", "LocalOriginUV": "1.036989,2.496478", "LowerLeftUV": "0.836989,2.296478", "RotationUV": "0", "ScanSignal": "A", "ScanSource": "SEM", "StageUV": "1.036989,2.496478", "UpperRightUV": "1.236989,2.696478"}


In [13]:
ffi = FFI()
# Definizione della funzione presente nella libreria C
ffi.cdef("""
    int write_custom_tag(const char* filename,
                          unsigned short tag_code,
                          const char* tag_name,
                          const char* tag_value);
""")

lib = ffi.dlopen('./libmetadata_add.so')  # carica la libreria compilata

file_tiff = './origin/2025_08_31_U2280_Electrodes_GePP_HR_3um.tif'
registry = json.load(open('tag_registry.json'))
instrument = 'RAITH_VELION'
if instrument not in registry:
    raise ('The instrument choosen does not appear in the registry.')
tag_code = registry[instrument]
# Converti le stringhe in bytes per CFFI
file_tiff_b = file_tiff.encode('utf-8')
tag_name_b = instrument.encode('utf-8')
tag_value_b = json_str.encode('utf-8')

with tf.TiffFile(file_tiff) as tiff:
    for count, page in enumerate(tiff.pages):
        if tag_code not in page.tags:
            res = lib.write_custom_tag(file_tiff_b, tag_code, tag_name_b, tag_value_b)
            if res == 0:
                print(f'Tag scritto con successo nella pagina {count}')
            else:
                print(f'Errore scrittura tag, codice: {res}')
        else:
            print(f'Tag {tag_code} già presente nella pagina {count}')

Tag scritto con successo nella pagina 0


TIFFFetchNormalTag: Warning, ASCII value for tag "Make" contains null byte in value; value incorrectly truncated during reading due to implementation limitations.


In [17]:
with tf.TiffFile('./outputs/2025_08_31_U2280_Electrodes_GePP_HR_3um.tif') as tiff:
    for page in tiff.pages:
        for tag in page.tags:
            print(f'{tag.name} con codice {tag.code} e valore {tag.value} trovato')

NewSubfileType con codice 254 e valore 0 trovato
ImageWidth con codice 256 e valore 512 trovato
ImageLength con codice 257 e valore 512 trovato
BitsPerSample con codice 258 e valore 8 trovato
Compression con codice 259 e valore 1 trovato
PhotometricInterpretation con codice 262 e valore 1 trovato
Make con codice 271 e valore Raith GmbH, D-44227 Dortmund trovato
StripOffsets con codice 273 e valore (266, 8458, 16650, 24842, 33034, 41226, 49418, 57610, 65802, 73994, 82186, 90378, 98570, 106762, 114954, 123146, 131338, 139530, 147722, 155914, 164106, 172298, 180490, 188682, 196874, 205066, 213258, 221450, 229642, 237834, 246026, 254218) trovato
Orientation con codice 274 e valore 1 trovato
SamplesPerPixel con codice 277 e valore 1 trovato
RowsPerStrip con codice 278 e valore 16 trovato
StripByteCounts con codice 279 e valore (8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 819